In [205]:
import numpy as np
from random import *

In [206]:
# taler = 1

learned = np.zeros([0,4])
obs = np.zeros([0,5])
epsilon = 0.1

In [217]:
spielfeld = np.zeros([10,10])
spielfeld[0,:] = 2
spielfeld[9,:] = 2
spielfeld[:,0] = 2
spielfeld[:,9] = 2
spielfeld[8,8] = 1
spielfeld[4,2] = 1
spielfeld[6,4] = 1
spielfeld[3,7] = 1
spielfeld[5,6] = 1
spielfeld = spielfeld.astype(int)
state = [1,1]

In [218]:
learned, obs = do_action(state, spielfeld, learned, epsilon, obs)
# 1. Richtung des nächsten Coins: (1,2,3,4:links,oben,rechts,unten)
# 2.-5. Direkte Umgebung: (links, oben, rechts, unten; 0=frei, 1 = Coin, 2 = Wand)
# Actions: "down", "up", "left", "right", "wait"
# [3. 2. 2. 0. 0.]

COIN COLLECTED 47
COIN COLLECTED 55
COIN COLLECTED 60
COIN COLLECTED 93
COIN COLLECTED 129
learned:
[[-1.14508584e+02 -1.16054924e+03 -7.00000000e+02 -7.00000000e-01]
 [-5.67406916e+02 -5.66220563e+02 -7.00000000e+02 -7.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.16815533e+02 -8.40616498e+02 -7.00000000e-01 -7.00000000e-01]
 [-6.81882196e+02 -4.61015369e+02 -7.00000000e-01 -7.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.19563274e+02 -1.39458725e+03 -7.00000000e-01 -7.00000000e-01]
 [-5.84484951e+02 -6.56038669e+02 -7.00000000e-01 -7.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-6.30082216e+02 -1.20744216e+03 -7.00000000e-01 -7.00000000e-01]
 [ 4.69191200e+02 -4.59193362e+02 -7.00000000e-01 -7.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.60000000e+02]
 [-1.08277169e+03 -1.41080466

In [168]:
def do_action(state, spielfeld, learned, epsilon, obs):
    actions = np.array(["down", "up", "left", "right"])
    discount = 0.7
    learning_rate = 0.8
    
    for i in range(1000):
        n_state = [0,0]
        if False:
            print("+++++++++++++++++++")
        else:
            indices = np.array([])
            possible_actions = np.array([])
            current_obs = get_observation(spielfeld,state[0],state[1])
            #print(state[0],state[1], current_obs)
            # print(current_obs)
            obs, index_current, learned = update_and_get_obs(obs, current_obs, learned)
            
            for j in range(actions.shape[0]):
                f_state = get_following_state(state, spielfeld, actions[j], actions, False)
                # print(f_state)
                new_obs = get_observation(spielfeld,f_state[0],f_state[1])
                obs, to_add, learned = update_and_get_obs(obs, new_obs, learned)
                indices = np.arange(actions.shape[0])
            
            possible_actions = actions
            indices = indices.astype(int)
            values = learned[indices,:]

            if epsilon > uniform(0,1):
                # epsilon -= 0.001
                # print("RANDOM")
                b = values.flatten()
                optima = np.random.choice(np.flatnonzero(b))
                best_choice = possible_actions[optima // actions.shape[0]]
                best_value = values.flatten()[optima]
            else:
                b = values.flatten()
                optima = np.random.choice(np.flatnonzero(b == b.max()))
                best_choice = possible_actions[optima // actions.shape[0]]
                best_value = values.flatten()[optima]

            n_state = get_following_state(state, spielfeld, best_choice, actions, True)
            action_index = optima // actions.shape[0]
                
            # print(best_choice)
            # print(state[0],state[1], best_choice)
            reward, spielfeld = get_reward(state,spielfeld,best_choice, i)
            learned[index_current, action_index] = (1-learning_rate) * learned[index_current,action_index] + learning_rate * (best_value + discount * reward)
            state = n_state
    print("learned:")
    print(learned)
    print("observations:")
    print(obs)
    return learned, obs

In [123]:
def get_following_state(state, spielfeld, choice, actions, wall_sensitive):
    n_state = np.array([0,0])
    if(choice == "down"):
        n_state[0] = state[0] + 1
        n_state[1] = state[1]
    elif(choice == "up"):
        n_state[0] = state[0] - 1
        n_state[1] = state[1]
    elif(choice == "left"):
        n_state[0] = state[0]
        n_state[1] = state[1] - 1
    elif(choice == "right"):
        n_state[0] = state[0]
        n_state[1] = state[1] + 1
    elif(choice == "wait"):
        n_state[0] = state[0]
        n_state[1] = state[1]
        
    if wall_sensitive and spielfeld[n_state[0], n_state[1]] == 2:
        # return old state if action is not possible
        return state
    
    return n_state

def get_reward(state,spielfeld,action, step):
    x,y = 0,0
    if action == "left":
        x = -1
    if action == "right":
        x = 1
    if action == "down":
        y = 1
    if action == "up":
        y = -1
    
    y +=state[0]
    x +=state[1] 
    
    copy = np.copy(spielfeld)
    copy[state[0], state[1]] = 8
    # print(copy)
    if spielfeld[y,x] == 1:
        # collect coin
        print("COIN COLLECTED " + str(step))
        spielfeld[y,x] = 0
        return 1000, spielfeld
    elif spielfeld[y,x] == 2:
        # Mauer
        return -1000, spielfeld
    else:
        return -1, spielfeld

In [124]:
def get_observation(spielfeld, y, x):
    # Suche den Coin, der am nächsten ist (manhattan distance)
    coinsY, coinsX = np.where(spielfeld == 1)
    distancesY = np.abs(coinsY - y)
    distancesX = np.abs(coinsX - x)
    observation = np.array([0, 0,0,0,0]) 
    # 1. Richtung des nächsten Coins: (1,2,3,4:links,oben,rechts,unten)
    # 2. Direkte Umgebung: (links, oben, rechts, unten; 0=frei, 1 = Coin, 2 = Wand)
 
    if not distancesY.shape[0] == 0:
        distances = distancesY + distancesX
        coinY = coinsY[np.argmin(distances)]
        coinX = coinsX[np.argmin(distances)]
        # setze observation flags:
        if coinX < x:
            observation[0] = 1
        elif coinX > x:
            observation[0] = 3
        elif coinY > y:
            observation[0] = 2
        elif coinY < y:
            observation[0] = 4
    
    
    observation[1] = get_spielfeld(y, x - 1,spielfeld)
    observation[2] = get_spielfeld(y - 1, x,spielfeld)
    observation[3] = get_spielfeld(y, x + 1,spielfeld)
    observation[4] = get_spielfeld(y + 1, x,spielfeld)
    # links, oben, rechts, unten
    # print(x,y, observation)
    return observation

def update_and_get_obs(db, new_obs, learned):
    temp = -1
    for i in range(db.shape[0]):
        if np.array_equal(db[i], new_obs):
            return db, i, learned
    db = np.append(db,np.array([new_obs]), axis = 0)
    learned = np.append(learned, np.zeros([1,learned.shape[1]]), axis = 0)
    return db, (db.shape[0] - 1), learned

def get_spielfeld(y,x,spielfeld):
    if y > spielfeld.shape[0] - 1 or x > spielfeld.shape[1] - 1 or y < 0 or x < 0:
        #print("collision")
        return 2
    else:
        return spielfeld[y,x]

In [29]:
get_spielfeld(1,0,spielfeld)

2

In [489]:
db, y, learned = update_and_get_obs(db, new,learned)

[[  1   3   2]
 [  4 455   6]
 [  2   6   8]]
[1 6 8]
new


In [490]:
db

array([[  1,   3,   2],
       [  4, 455,   6],
       [  2,   6,   8],
       [  1,   6,   8]])

In [491]:
learned

array([[2., 3., 6., 6.],
       [4., 3., 6., 6.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [94]:
get_observation(spielfeld,0,8)

collision
8 0 [0 2 2 2 0]


array([0, 2, 2, 2, 0])